In [27]:
# First import the libraries that we need to use
import pandas as pd
import requests
import json
# from dotenv import load_dotenv
# load_dotenv()
    

True

In [44]:
# def get_currencies():
coin_url = "https://api.exchange.coinbase.com/currencies"
coin_response = requests.get(coin_url)
if coin_response.status_code == 200:
    data = pd.DataFrame(json.loads(coin_response.text), columns=['id', 'name', 'min_size', 'status', 'max_precision']).fillna('')
    data.to_csv("coinbase_currencies.csv", index=False)
data.head(999)

,id,name,min_size,status,max_precision
0,DOT,Polkadot,0.01,online,0.001
1,ALGO,Algorand,1,online,0.000001
2,ATOM,Cosmos,1,online,0.000001
3,ACH,Alchemy Pay,0.1,online,0.00000001
4,1INCH,1Inch,0.1,online,0.01
...,...,...,...,...,...
173,GODS,Gods Unchained,0.1,online,0.00000001
174,GTC,Gitcoin,0.1,online,0.01
175,ICP,Internet Computer,0.01,online,0.001
176,NU,NuCypher,10,online,0.000001


In [45]:
## Fetch data and save to CSV file

def fetch_daily_data(symbol):
    pair_split = symbol.split('/')  # symbol must be in format XXX/XXX ie. BTC/EUR
    symbol = pair_split[0] + '-' + pair_split[1]
    # url = f'https://api.pro.coinbase.com/products/{symbol}/candles?granularity=86400'
    url = f'https://api-public.sandbox.exchange.coinbase.com/products/{symbol}/candles?granularity=86400'
    
    response = requests.get(url)
    if response.status_code == 200:
        data = pd.DataFrame(json.loads(response.text), columns=['unix', 'low', 'high', 'open', 'close', 'volume'])
        data['date'] = pd.to_datetime(data['unix'], unit='s')  # convert to a readable date
        data['vol_fiat'] = data['volume'] * data['close'] # multiply the BTC volume by closing price to approximate fiat volume

        if data is None:
            print("Did not return any data from Coinbase for this symbol")
        else:
            filename = f'coinbase_dailydata_{pair_split[0] + pair_split[1]}.csv'
            data.to_csv(filename, index=False)
            print(f'Created CSV file: {filename}.')
    else:
        print("Did not receieve OK response from Coinbase API")

if __name__ == "__main__":
    pair = "XRP/USD"
    fetch_daily_data(symbol=pair)


Created CSV file: coinbase_dailydata_XRPUSD.csv.


In [48]:
## Get product information
product_url = "https://api.exchange.coinbase.com/products"
headers = {"Accept": "application/json"}
product_response = requests.get(product_url, headers=headers)
product_columns=["id","base_currency","quote_currency","base_min_size","base_max_size","quote_increment","base_increment","display_name","min_market_funds","max_market_funds","margin_enabled","fx_stablecoin","max_slippage_percentage","post_only","limit_only","cancel_only","trading_disabled","status","status_message","auction_mode"]
product_df = pd.DataFrame(json.loads(product_response.text), columns=product_columns)

# product_df


In [50]:
## Get trades for ...

def fetch_trade_data(symbol):
    pair_split = symbol.split('/')
    symbol = pair_split[0] + '-' + pair_split[1]
    trades_url = f"https://api.exchange.coinbase.com/products/{symbol}/trades"
    headers = {"Accept": "application/json"}
    trades_columns = ["time","trade_id","price","size","side"]
    response = requests.get(trades_url, headers=headers)

    if response.status_code == 200:
        trades_data = pd.DataFrame(json.loads(response.text), columns=trades_columns)
        trades_data['date'] = pd.to_datetime(trades_data['time'])  # convert to a readable date

        if trades_data is None:
            print("Did not return any data from Coinbase for this symbol")
        else:
            filename = f'coinbase_trades_{pair_split[0] + pair_split[1]}.csv'
            trades_data.to_csv(filename, index=False)
            print(f'Created CSV file: {filename}.')
    else:
        print("Did not receieve OK response from Coinbase API")

if __name__ == "__main__":
    pair = "XRP/USD"
    fetch_trade_data(symbol=pair)


Created CSV file: coinbase_trades_XRPUSD.csv.
